# 09 - Classification of Roof Superstructures

The notebook shows a multi-class classification with support vector machines on the application example of classifying roof superstructures from ALS data. For the application context, please refer to the respective slides in ISIS.

To come straight to the point, the accuracies that can be achieved with the data are not particularly high. But one should not forget that the corresponding point subsets of the 3D point clouds could not be classified at all with conventional, empirical rule-bases approaches. Accordingly, the methodology is definitely an improvement.

In addition to learning multi-class classification, which is not difficult at all from a programming point of view in scikit learn, the objective of this notebook is also the own critical interpretation of the results.

The data records of the dataset stores geometric features computed for groups of points from an ALS 3D point cloud.

**Description of Features:**
1. **num_pts**: numper of points
2. **max_height_diff**: maximum height difference between the highest and the lowest point in the segment
3. **height_diff_to_build**: height difference between the highest point in building, which the segment belongs to, and the highest point in segment (Chimneys are typically the highest roof superstructure, and their value for this feature should then be 0.)
4. **area_from_conv_hull**: area computed from convex hull of segment
5. **area_from_alpha_shp**: area computed from alpha shape (The alpha-shape is something like a concave hull of the segment. Maybe check the figure in the wikipedia article: https://en.wikipedia.org/wiki/Alpha_shape)
6. **min_elevation**: minimum elevation of the segment
7. **max_elevation**: maximum elevation of the segment
8. **avg_elevation**: average elevation of the segment
9. **size_entropy**: (in short) the size of the segment in relation to the size of the building to which the segment belongs
10. **elevation_entropy**: (in short) quantifies the degree of uncertainty provided by the heights of points constituting the segment
11. **std_height**: standard deviation of the height values of the points
12. **coef_variation**: coefficient variation of heights

The **target classes** are 'shed dormer', 'gable dormer','ground', 'chimney', 'others' given as numeric values 1, 2, ..., 5. The class 'ground' seems out of place, since it is no roof superstructure. But some ground points in the dataset that are located close to a building were misclassified as building points (in the point cloud classification step) and they need now to be identified as ground. The class 'others' contains group of points that are superstructures, but which could not be identified by a human during the labeling process.

Please note that those code parts that are mainly a repetition of previous notebooks are no longer greatly described and documented. Refer to the previous notebooks if something is unclear.

## Load Data

In [1]:
from pathlib import Path
import os

data_dir = str(Path.home()) + r'/coursematerial/GIS/GeoDataScience'

filepath = os.path.join(data_dir, r'RoofSuperstructures.csv')

print(filepath)

/home/jovyan/coursematerial/GIS/GeoDataScience/RoofSuperstructures.csv


In [2]:
import pandas as pd

column_names = ['class', 'num_pts', 'max_height_diff', 'height_diff_to_build', 'area_from_conv_hull', 'area_from_alpha_shp', 
                'min_elevation', 'max_elevation', 'avg_elevation', 'size_entropy', 'elevation_entropy', 'std_height', 'coef_variation', 'building_id', 'segment_id']

data_df = pd.read_csv(filepath, sep=' ', names=column_names)

In [3]:
data_df.head()

,class,num_pts,max_height_diff,height_diff_to_build,area_from_conv_hull,area_from_alpha_shp,min_elevation,max_elevation,avg_elevation,size_entropy,elevation_entropy,std_height,coef_variation,building_id,segment_id
0,1,7,0.27,7.84,0.389893,0.342041,288.55,288.82,288.692857,0.127143,2.514573,0.142857,0.038571,0,8
1,3,13,0.23,6.97,1.392822,0.815674,289.46,289.69,289.588462,0.101538,1.621488,0.128462,0.017692,0,10
2,5,6,0.28,1.84,0.081787,0.002197,293.84,294.12,293.981667,0.138333,0.584963,0.141667,0.046667,1,4
3,3,7,0.13,1.87,0.552734,0.097656,294.91,295.04,294.981429,0.058571,2.584963,0.071429,0.018571,2,5
4,5,6,0.97,0.95,0.057129,0.066406,294.99,295.96,295.475000,0.485000,2.807355,0.485000,0.161667,2,6


## Shuffle & Split

The class **ShuffleSplit** allows to generate random permutations for given datasets that can be used for cross validation. (But be aware that the returned folds are not disjoint, meaning that data records may appear in several folds.) In the following, we provide the size of the test data (as a fraction), and some random state number to have repeatable results. The constructed **ShuffleSplit** object can then be used with the **split()** method on some data to get the indices for training and testing.

In [4]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(test_size=0.30, random_state=0)
ss_gen = ss.split(data_df)
type(ss_gen)

generator

The **split()** method does not give the indices directly, but returns a generator object that can be used as an iterator to iterate through the folds, and retrieve the indices per fold. (By default, the number of splits is 10. But this can also be changed with the **n_splits** parameter of the **ShuffleSplit** constructor.

(Because ss_gen is a generator, you can only iterate through it once. If you want to repeat the following for-loop, then you need to construct ss_gen once more.)

In [6]:
for train_idx, test_idx in ss_gen:
    print(train_idx[0:10], ' ', test_idx[0:5])

[  82 1327  438  998  349 1015  371  117 1337  252]   [ 317  587 1158  610 1241]
[1061   27  674  647  559  509  424  628 1208  397]   [1363  812 1338 1045  446]
[1045  161  908  347  712 1075  873 1102  780  412]   [ 848  733 1323  556 1085]
[1097  113 1151  750  607  824  963  973  645  460]   [1169  806  878 1025 1143]
[1039  816  852 1036 1318  722   19 1285  491 1162]   [  77  744  976 1159  844]
[ 471  303  252  938  267 1127  341  921  574  996]   [ 807 1041 1380 1337  245]
[ 349  105   71 1014   19   82  367  718  903 1180]   [118  90 334 640 919]
[1205 1047  428  293  814  406 1263   48  638  900]   [163 246 310 612 884]
[1049  863  872  430  811  631  892  635  530  353]   [ 375 1231  760  927  131]
[ 432 1145  597 1014 1200 1354 1158  879  466  178]   [ 787 1101 1179  148  883]


The for-loop uses the **next()** method of the generator object to get the data for the next iteration. (Every iterator and generator needs to provide this method, so that the object can be used in a for-loop.) If we just want one fold, then we can call the **next()** method directly on the result of the **split()** method. One fold of a training and testing split can be retrieved as a one-liner of code with the following:

In [7]:
train_idx, test_idx = next(ShuffleSplit(test_size=0.30, random_state=0).split(data_df))

Now, we can extract the data records of training and testing using slicing on the data frame object.

(Remember that we can use numeric indices starting with 0 when using **iloc** on rows and columns. Otherwise, we would need to work with the row indices and the column names of the data frame.)

In [8]:
train_df = data_df.iloc[train_idx, :]
test_df = data_df.iloc[test_idx, :]

And we can extract the columns for input features and target labels for both training and testing data, again with slicing. (Of course, we could slice the rows and columns in one step.)

In [10]:
X_train = train_df.iloc[:, 1:13]
X_train.head()

,num_pts,max_height_diff,height_diff_to_build,area_from_conv_hull,area_from_alpha_shp,min_elevation,max_elevation,avg_elevation,size_entropy,elevation_entropy,std_height,coef_variation
82,9,1.360,7.060,0.664795,0.555420,297.540,298.900,297.807778,1.092222,2.222392,0.267778,0.151111
1327,89,3.530,0.870,60.175537,2.346436,328.370,331.900,330.243146,1.656854,0.344446,1.873146,0.039663
438,16,2.215,-1.885,2.134766,0.968750,277.340,279.555,278.617813,0.937188,2.409391,1.277812,0.138438
998,10,1.201,5.823,1.083496,0.176270,273.494,274.695,274.188200,0.506800,3.378512,0.694200,0.120100
349,5,0.588,-1.190,0.196045,0.188965,277.102,277.690,277.482200,0.207800,3.104337,0.380200,0.117600


In [11]:
y_train = train_df.iloc[:, 0:1]
y_train.head()

,class
82,5
1327,5
438,1
998,1
349,4


In [12]:
X_test = test_df.iloc[:, 1:13]
y_test = test_df.iloc[:, 0:1]

## Explore Class Balance

Next, we take a look at the class balance of the data. First for the whole dataset, then for the training and testing. 

In all cases, we want a sorted output of the counted occurrences of the class labels (of y).

For the whole dataset, we need to first slice the class column using iloc and numeric indices, convert the data frame into a NumPy array with the method **to_numpy()**, and get rid of the second dimension with **flatten()**. (When converting a data frame into a NumPy array, you end up with a 2D array.) The result (a 1D array) can be given to the **Counter** class. 

The **Counter** class will count the occurrences of every value it finds in the given data collection. And the method **items()** returns a dictionary of values found, and the number of occurrences.

In [16]:
from collections import Counter
c = Counter(data_df.iloc[:, 0:1].to_numpy().flatten())
for i in c.items():
    print(i)

(1, 241)
(3, 47)
(5, 869)
(4, 155)
(2, 100)


To get a sorted output, we can use the **sorted()** function, which takes an iterable (any collection that can be used as an iterator (and which provides the **next()** method)) and makes a sorted list from the values the iterable provides. This list can then be used in a for-loop to print its content. To access the first and second value of the items returned by **items()**, we can use indexing with [0] and [1].

A sorted output of classes according to their number of occurrences is then:

In [17]:
for i in sorted(Counter(data_df.iloc[:, 0:1].to_numpy().flatten()).items()):
    print(i[0], ':', i[1])


1 : 241
2 : 100
3 : 47
4 : 155
5 : 869


The same for the training labels.

In [18]:
for i in sorted(Counter(y_train.to_numpy().flatten()).items()):
    print(i[0], ':', i[1])

1 : 171
2 : 75
3 : 38
4 : 109
5 : 595


And for the testing labels. (We could define a function for convenience, so that we do not need to type (or copy & paste) the same code over and over again.)

In [19]:
for i in sorted(Counter(y_test.to_numpy().flatten()).items()):
    print(i[0], ':', i[1])

1 : 70
2 : 25
3 : 9
4 : 46
5 : 274


The split seems all right on first sight, and we get data records with the same class distribution in the whole dataset, the training subset, and the testing subset. However, class 5, which is the class for 'others', is highly overrepresented. This might pose a problem as these groups of points could not be identified by a human as a valid roof superstructure, but could end up having the same geometric properties as the other roof superstructures.

## Support Vector Machine Model

Next, we construct a SVM object and train it with the training data. Be careful that y_train is still a data frame and the **fit()** method of the support vector classifier wants a 1D array. So, we need to extract the class column.

The hyperparameters (gamme and C) have been empirically determined. Once you got to the end of the notebook, you are asked to find better ones.

In [20]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf', gamma=1, C=32768)
svc.fit(X_train, y_train['class'])

SVC(C=32768, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

With the method **score()** of the **SVC** class, you can get the (mean) accuracy of the classifier giving the test data.

In [21]:
svc.score(X_test, y_test['class'])

0.6556603773584906

Now, lets predict the classes from the test data using the **predict()** method.

In [22]:
y_pred = svc.predict(X_test)


Which we can now use to get a classification report.

In [23]:
from sklearn.metrics import classification_report
class_names = 'shed dormer', 'gable dormer','ground', 'chimney', 'others'
print(classification_report(y_test['class'], y_pred, target_names=class_names))

              precision    recall  f1-score   support

 shed dormer       1.00      0.04      0.08        70
gable dormer       1.00      0.08      0.15        25
      ground       0.00      0.00      0.00         9
     chimney       0.00      0.00      0.00        46
      others       0.65      1.00      0.79       274

    accuracy                           0.66       424
   macro avg       0.53      0.22      0.20       424
weighted avg       0.65      0.66      0.53       424



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Take a look at the provided slides on ISIS that give some definitions of precision, recall, and f1-score.

(Remember that all metrics are between 0 and 1, and that 0 (0%) means (simplified) bad, and 1 (100%) means good.)

The **precision** gives us the proportion of the items that we correctly classified to belong to a certain class in comparison to the overall items that we identified to belong to this class. Both 'shed dormer' and 'gable dormer' have a high precision (1.0), which means that all data records that we identified as 'shed dormer' or 'gable dormer' are actually items of these classes. This seems to be a great result, because we actually wanted to identify roof superstructures. Although chimneys are not identified at all, but they also seem to not be many in the dataset anyway. And since we are not interested in ground and others, we can care less about those. **Is this a correct interpretation of the results?**

To answer this question, we also have to take a look at **recall**. This metric gives us the proportion of the data records that belong to a class were actually identified correctly. So, it seems that we only identified 4% and 8% of the 'shed dormer' and 'gable dormer' objects in the dataset. So, our model missed almost all of the roof superstructures. The only class it got correct in this metric was others. All objects that are in reality 'others' are also classified as 'others'.

The **f1-score** brings both precision and recall together, and we can see that only the class 'other's has a relevant value for the f1-score.

And because the class 'others' is overrepresented, the overall (mean) accuracy is still high with 66%.

To get more insight, lets us also take a look at the confusion matrix.

In [24]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(cm, index=class_names, columns=class_names)

,shed dormer,gable dormer,ground,chimney,others
shed dormer,3,0,0,0,67
gable dormer,0,2,0,0,23
ground,0,0,0,0,9
chimney,0,0,0,0,46
others,0,0,0,1,273


Take yourself some time and interpret the confusion matrix with what is given in the classification report and what is written above.

What can be said as a final remark is that this classifier just classifies everything as 'others'. And because most of the data records are 'others', the accuracy is pretty high.

## A Better SVM Pipeline

As mentioned in the lecture video, SVMs are highly sensitive to unnormalized data. Therefore, we better normalize our features. This can be accomplished, e.g., with the **StandardScaler** class, which we apply to our data before feeding it to the support vector machine classifier class **SVC**. Because we need to apply the scaler to all our data that we feed to the **fit()** method, and also to the **predict()** method, we can build a pipeline that combines several classes together. Our pipeline is then the **StandardScaler** and the **SVC** class. All calls to **fit()** and **predict()** will go through all steps in the pipeline.

The pipeline constructor takes a list of tuples, where the first element of the tuple gives the name of the pipeline step, and the second element of the tuple is the object to be called in the pipeline.

In [92]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
svm_pipeline = Pipeline([
('scaler', StandardScaler()),
('svm_cls', SVC(kernel='rbf', gamma=0.2, C=32768000))
])


We can use this pipeline in the same way as a stand-alone machine learning algorithm like SVC, e.g. to fit the model to the data.

In [93]:
svm_pipeline.fit(X_train, y_train['class'])

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_cls',
                 SVC(C=32768000, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=0.2,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

Get the score (accuracy) for this model.

In [94]:
svm_pipeline.score(X_test, y_test['class'])




0.535377358490566

Not as good as before. But better check the classification report from the predictions.

In [95]:
y_pred = svm_pipeline.predict(X_test)

In [96]:
class_names = 'shed dormer', 'gable dormer','ground', 'chimney', 'others'
print(classification_report(y_test['class'], y_pred, target_names=class_names))



              precision    recall  f1-score   support

 shed dormer       0.32      0.47      0.38        70
gable dormer       0.30      0.48      0.37        25
      ground       0.16      0.33      0.21         9
     chimney       0.24      0.20      0.21        46
      others       0.76      0.62      0.68       274

    accuracy                           0.54       424
   macro avg       0.35      0.42      0.37       424
weighted avg       0.59      0.54      0.55       424



Although the overall accuracy is a little lower, the precision, recall, and f1-score increases for (almost) all classes.

Taking a look at the confusion matrix.

In [97]:
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(cm, index=class_names, columns=class_names)

,shed dormer,gable dormer,ground,chimney,others
shed dormer,33,4,1,7,25
gable dormer,5,12,0,1,7
ground,3,0,3,1,2
chimney,13,2,3,9,19
others,50,22,12,20,170


Now, the confusion matrix actually has some values in the diagonal, which is very important. These are the correctly classified data records.

The method is still rather weak with around 60%. One the one hand, 25 shed dormers were correctly identified. But on the other hand, more data records of the class 'others' were classified as shed dormers than there are actually shed dormers.

Take a look on the classification report and confusion matrix and make sure you understand it.

As final remarks:

- The class 'others' might have a bad influence on the classifier, because these groups of points cannot be identified by a human. And they might as well be shed dormers or something else. Most likely, they are a mix of all kinds of superstructures, and maybe even some high vegetation. 

- If we would use this method to model further superstructures, we could very likely throw everything out that is predicted to belong to the predicted classes of 'others' and 'ground'. And we would end up with more superstructures on the 3D building models than before with around 40% of them being right.

- We could also train the classifier to give output probabilities instead of class labels, and then use the method **predict_proba()** to get the probabilities of all classes instead of a class. And if the probability is low, we could ignore this prediction. Maybe this would give us a higher accuracy for the relevant classes.

- We could address the imbalance of the classes. However, our experiments did not really work out well with this dataset.

## Your Task:

Try out different values for the hyperparameters gamma and C and observe how the overall accuracy (score), the classification report, and the confusion matrix develops. **It is important to develop a good intuition on how to interpret the quality of the model.**